In [15]:
import os
import pandas as pd
import numpy as np
import ee

In [16]:
# 认证并初始化Google Earth Engine
ee.Authenticate()  # 只需要在第一次运行时执行
ee.Initialize(project= 'ee-zongrong-flood')

In [17]:
# Define the filtered dataset
dataset = ee.ImageCollection('WorldPop/GP/100m/pop').filterDate('2020-01-01', '2020-12-31').mosaic()

# Calculate the min and max values for the filtered dataset
min_max = dataset.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=ee.Geometry.BBox(-180, -90, 180, 90),
    scale=100,
    bestEffort=True
).getInfo()

min_value = min_max['population_min']
max_value = min_max['population_max']

print(f"Min value: {min_value}, Max value: {max_value}")

Min value: 0, Max value: 839.478271484375


In [19]:
# Function to get population value at given coordinates and normalize it
def get_population_value(coords):
    point = ee.Geometry.Point(coords)
    population_value = dataset.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=100
    ).get('population')
    
    value = population_value.getInfo()
    if value is None:
        return 0
    
    # Apply logarithmic transformation before normalization
    log_value = np.log(value + 1)  # Adding 1 to avoid log(0)
    log_min_value = np.log(min_value + 1)
    log_max_value = np.log(max_value + 1)
    
    normalized_value = (log_value - log_min_value) / (log_max_value - log_min_value) * 10
    print(f"Coordinates: {coords}, Original value: {value}, Normalized value: {normalized_value}")  # Debugging line
    return normalized_value

# Specify the input and output directories
input_dir = 'C:/Users/lzr/SSCI/StreetviewLLM/groudtruth/Sampling'
output_dir = 'C:/Users/lzr/SSCI/StreetviewLLM/groudtruth/pop'

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Process each CSV file in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith('.csv'):
        input_csv = os.path.join(input_dir, filename)
        df = pd.read_csv(input_csv)
        
        # Create a new column for population values
        df['Population'] = df.apply(lambda row: get_population_value([row['longitude'], row['latitude']]), axis=1)
        
        # Define the output filename
        output_csv = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_pop.csv")
        
        # Save the new dataframe to a CSV file
        df.to_csv(output_csv, index=False)
        
        print(f"Processed {input_csv} and saved with population values as {output_csv}")

print("All CSV files processed.")

Coordinates: [114.276307, 22.2695341], Original value: 2.4703924655914307, Normalized value: 1.8477472971693423
Coordinates: [114.1023043902492, 22.37115357347453], Original value: 60.92076110839844, Normalized value: 6.126927869689128
Coordinates: [114.2983256, 22.2618995], Original value: 0.9477043747901917, Normalized value: 0.9899826200932871
Coordinates: [114.18624955824998, 22.34541677866001], Original value: 415.8418884277344, Normalized value: 8.958617278441622
Coordinates: [114.1706041480453, 22.26166046887501], Original value: 4.3063859939575195, Normalized value: 2.478345948959175
Coordinates: [114.25291622465, 22.30375011216001], Original value: 420.7599182128906, Normalized value: 8.97603531057199
Coordinates: [114.21958289145, 22.32041677876001], Original value: 493.30804443359375, Normalized value: 9.211739702896532
Coordinates: [114.16958289165, 22.34541677866001], Original value: 51.33552932739258, Normalized value: 5.877179183474928
Coordinates: [114.26958289125002, 2

Coordinates: [114.2041615919109, 22.31957680366856], Original value: 41.910888671875, Normalized value: 5.582331068478046
Coordinates: [114.3082692, 22.3633873], Original value: 0.5949339270591736, Normalized value: 0.6932496492216524
Coordinates: [114.2240295, 22.36166], Original value: 1.1926988363265991, Normalized value: 1.165928868325895
Coordinates: [114.275856, 22.2958202], Original value: 7.6224846839904785, Normalized value: 3.199261264046995
Coordinates: [114.20291622485, 22.287083445560008], Original value: 89.27096557617188, Normalized value: 6.686716958692398
Coordinates: [114.234072, 22.3646607], Original value: 0.974322497844696, Normalized value: 1.0101399163606675
Coordinates: [114.16124955835, 22.26208344566001], Original value: 7.669229984283447, Normalized value: 3.2072902315256897
Coordinates: [114.12791622515, 22.32041677876001], Original value: 37.72515106201172, Normalized value: 5.429915324181844
Coordinates: [114.21958289145, 22.278750112260006], Original valu

Coordinates: [114.3023995, 22.279642], Original value: 0.6439812779426575, Normalized value: 0.7382284554361755
Coordinates: [114.22791622475, 22.26208344566001], Original value: 66.84164428710938, Normalized value: 6.262539838525727
Coordinates: [114.26124955795, 22.32041677876001], Original value: 114.83643341064453, Normalized value: 7.0570233644112355
Coordinates: [114.21124955815004, 22.25375011236001], Original value: 0.5809247493743896, Normalized value: 0.6801483900021756
Coordinates: [114.12791622515, 22.34541677866001], Original value: 267.42034912109375, Normalized value: 8.304986960969481
Coordinates: [114.0943324713332, 22.36997100052521], Original value: 83.45246124267578, Normalized value: 6.58777520859392
Coordinates: [114.2454502477145, 22.24532179272471], Original value: 5.358631610870361, Normalized value: 2.746987135879395
Coordinates: [114.2277734841905, 22.35528189321145], Original value: 1.7921078205108643, Normalized value: 1.5248013192707652
Coordinates: [114.2

Coordinates: [114.12791622515, 22.278750112260006], Original value: 179.25099182128906, Normalized value: 7.713650980988103
Coordinates: [114.1502728253321, 22.37057904753004], Original value: 4.598334312438965, Normalized value: 2.5578801624636704
Coordinates: [114.19458289155, 22.32041677876001], Original value: 85.85108947753906, Normalized value: 6.629364709561356
Coordinates: [114.2123717, 22.3459633], Original value: 39.303016662597656, Normalized value: 5.489222133669376
Coordinates: [114.26124955795, 22.35375011196001], Original value: 4.852000713348389, Normalized value: 2.623687536902097
Coordinates: [114.2112945569925, 22.3081724556816], Original value: 43.758724212646484, Normalized value: 5.644940145111591
Coordinates: [114.16124955835, 22.287083445560008], Original value: 82.4976806640625, Normalized value: 6.5708907699114025
Coordinates: [114.18624955824998, 22.32041677876001], Original value: 1072.192138671875, Normalized value: 10.362968134436052
Coordinates: [114.2852

Coordinates: [114.2736765, 22.2707772], Original value: 3.401054859161377, Normalized value: 2.2005503599096463
Coordinates: [114.2460516624531, 22.3431892446759], Original value: 9.777983665466309, Normalized value: 3.5306143558390075
Coordinates: [114.15291622505003, 22.27041677896001], Original value: 20.706911087036133, Normalized value: 4.570305763312008
Coordinates: [114.16958289165, 22.29541677886001], Original value: 138.30960083007812, Normalized value: 7.331036511163935
Coordinates: [114.26958289125002, 22.33708344536001], Original value: 3.061850070953369, Normalized value: 2.0814442642815294
Coordinates: [114.16124955835, 22.31208344546001], Original value: 282.7104187011719, Normalized value: 8.387256168918837
Coordinates: [114.23624955805002, 22.27041677896001], Original value: 172.45579528808594, Normalized value: 7.656585850579191
Coordinates: [114.1949411, 22.3614018], Original value: 4.152727127075195, Normalized value: 2.4347091623331667
Coordinates: [114.19458289155

...

Coordinates: [140.2703686620768, 35.81960271135796], Original value: 1.7704678773880005, Normalized value: 1.5132470863232022
Coordinates: [139.58624945665002, 35.853750057960006], Original value: 15.259175300598145, Normalized value: 4.141178127502482
Coordinates: [140.20291612085003, 35.71208339186001], Original value: 4.2320051193237305, Normalized value: 2.457382973421995
Coordinates: [140.28624945385002, 35.86208339126001], Original value: 3.14684796333313, Normalized value: 2.1121987271624096
Coordinates: [140.13624945445002, 35.70375005856001], Original value: 28.323694229125977, Normalized value: 5.016944408433907
Coordinates: [139.30291612445, 35.52875005926001], Original value: 3.9524519443511963, Normalized value: 2.375838524291395
Coordinates: [140.16124945435, 35.82041672476001], Original value: 2.6944730281829834, Normalized value: 1.940664590269453
Coordinates: [139.46124945715002, 35.89541672446001], Original value: 59.414791107177734, Normalized value: 6.09036465936526

Coordinates: [139.99458278835, 35.69541672526001], Original value: 78.11849975585938, Normalized value: 6.490890234294659
Coordinates: [139.70291612285, 35.60375005896001], Original value: 141.29071044921875, Normalized value: 7.362479207927959
Coordinates: [140.11958278785, 35.61208339226001], Original value: 85.3231201171875, Normalized value: 6.620309769105981
Coordinates: [139.4947045, 35.6550792], Original value: 29.713176727294922, Normalized value: 5.085694192466846
Coordinates: [139.27791612455002, 35.69541672526001], Original value: 20.528169631958008, Normalized value: 4.558027124153755
Coordinates: [139.71124945615003, 35.80375005816001], Original value: 95.2628402709961, Normalized value: 6.782153207133989
Coordinates: [139.58624945665002, 35.58708339236001], Original value: 125.94881439208984, Normalized value: 7.1930572563452975
Coordinates: [139.84458278895, 35.67875005866001], Original value: 33.14118576049805, Normalized value: 5.242826849569436
Coordinates: [140.17732

Coordinates: [139.3617465551039, 35.62078796325303], Original value: 29.656282424926758, Normalized value: 5.082940754145414
Coordinates: [140.32791612034998, 35.47041672616001], Original value: 1.9407098293304443, Normalized value: 1.601805197318307
Coordinates: [139.35291612425, 35.65375005876001], Original value: 55.84385681152344, Normalized value: 5.999889358569812
Coordinates: [139.7052734289497, 35.79701154579502], Original value: 13.740567207336426, Normalized value: 3.995567142537415
Coordinates: [140.31958278705002, 35.77041672496001], Original value: 35.622920989990234, Normalized value: 5.347029631022659
Coordinates: [140.18624945425, 35.83708339136001], Original value: 2.7201387882232666, Normalized value: 1.950945373273013
Coordinates: [139.48624945705, 35.46208339286001], Original value: 90.05004119873047, Normalized value: 6.6994781997824004
Coordinates: [140.09458278795, 35.62875005886001], Original value: 78.23016357421875, Normalized value: 6.492984621095501
Coordina

Coordinates: [139.45291612385, 35.59541672566001], Original value: 41.71276092529297, Normalized value: 5.575458626411729
Coordinates: [139.97791612175, 35.68708339196001], Original value: 67.79217529296875, Normalized value: 6.28320190871287
Coordinates: [140.14458278774998, 35.70375005856001], Original value: 20.804494857788086, Normalized value: 4.576966679280775
Coordinates: [139.3198151149338, 35.68822171173645], Original value: 15.6443510055542, Normalized value: 4.175947353936341
Coordinates: [139.97791612175, 35.71208339186001], Original value: 105.08187866210938, Normalized value: 6.926390337064866
Coordinates: [139.62791612315, 35.55375005916001], Original value: 97.62689208984375, Normalized value: 6.818181864852077
Coordinates: [140.02791612155, 35.72041672516001], Original value: 50.9527587890625, Normalized value: 5.866278250686818
Coordinates: [139.3124532095746, 35.51076482627948], Original value: 4.543830394744873, Normalized value: 2.5433516765210795
Coordinates: [139

Coordinates: [139.60291612325, 35.57041672576001], Original value: 46.787715911865234, Normalized value: 5.742181781595857
Coordinates: [139.48624945705, 35.43708339296001], Original value: 20.107229232788086, Normalized value: 4.528703129106054
Coordinates: [140.02791612155, 35.65375005876001], Original value: 57.39739990234375, Normalized value: 6.039929940072351
Coordinates: [139.35291612425, 35.50375005936001], Original value: 36.13301467895508, Normalized value: 5.367570492094464
Coordinates: [139.3634836444438, 35.74525063594564], Original value: 12.790460586547852, Normalized value: 3.8966265997856575
Coordinates: [139.47791612375, 35.81208339146001], Original value: 38.93004608154297, Normalized value: 5.475415650049423
Coordinates: [139.4709988687209, 35.8042646497295], Original value: 16.03940773010254, Normalized value: 4.210782494911652
Coordinates: [139.335352068147, 35.86895090869222], Original value: 7.666769981384277, Normalized value: 3.2068687820947095
Coordinates: [1